In [1]:
import cv2, time

In [ ]:
input_path = "C:\\Users\\Ajay\\Employee behaviour\\"

# Display Image in a Window

In [ ]:
#create a cascade Classifier 
#It has feature XML file as input i.e. XML file containt face feature
face_cascade = cv2.CascadeClassifier(input_path+"haarcascade_frontalface_default.xml")

#Reading the image as:

#Colored image parameter as 0
img = cv2.imread(input_path + "sample4.jpg",1)

#Gray scaled image parameter as 0
#img = cv2.imread(input_path + "sample1.jpg",0)

#Reading image as gray scaled image
gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

#Using detectMultiScale method to Search the rectangular coordinate of FACE in image; to figure out where is the face
faces = face_cascade.detectMultiScale(gray_img,  scaleFactor = 1.05, minNeighbors=5)

In [ ]:
print(img) #3D matrix for colored image and 2D for gray scaled
print(type(img)) #image will be stored as numpy array
print(img.shape) #1174 ROW, 1300 COLUMN, 3 Channel

In [ ]:
#to add rectangular FACE BOX to face use rectangle() mehtod
for x,y,w,h in faces:
    img = cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 3)


#to display the image in window use imshow()
cv2.imshow("ajay", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Capture first Image from video

In [ ]:
#video is multiple frames displayed very quickly
video = cv2.VideoCapture(0) #0 is for internal webcam and 1 is for external

#Show the camera in a Window
check, frame = video.read() 
print(check)#check is bool type it return whether python is able to read the VideoCapture object
print(frame)#frame is a numy array (image), it represent first image that video captures

time.sleep(10) #Holf the camera for 10 Secs
cv2.imshow('Capturing Camera',frame)
cv2.waitKey(0)
video.release()
cv2.destroyAllWindows()

# Capture entire video with help of loop

To capture video we will use while loop. Until, 'CHECK' is true , python will display the frame

In [ ]:
#video is multiple frames displayed very quickly
video = cv2.VideoCapture(0) #0 is for internal webcam and 1 is for external

a=1

while True:
    a= a+1
    check,frame = video.read()
    print(frame) #It is RGB image
    
    gray_img = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) # It is GRAY Image
    
    cv2.imshow('Capturing Camera',gray_img)
                          #If we specify 0 , the movement we press a key the window closes
    key = cv2.waitKey(1) #We are generating the new frame after every 1ms sec
                            #After every 1ms frame Window will distroy
    
    if key == ord('q'):     #If user press Q from keyboard then terminate the loop
        break
    
print(a) #print no of frame we have captured
video.release() #Free the camera resource
cv2.destroyAllWindows() #Kill all window

# Motion Detection

Organisation that study human behaviour. Our task is to give them how long a person was infront of a camera

#STEP1 : Save the initial image in a FRAME (Image w/o Object)
#STEP2 : Convert this image in a Gaussian blur image
#STEP3 : Take a next frame that appear after 1st frame (Image w/ object) and also convet it in a gaussian blur image
#STEP4 : Calculate the difference. This difference will tell us whether there is a object infront of camera or not. If there is change from 1st frame to 2nd frame then there is definitely a object. But that object might be too small and I don't want these object.
#STEP5 : Define the threshold to remove the showdow(mosquito) and other noise
#STEP6 : Define the border object
#STEP7 : Add the rectangular box around the object
#STEP8 : Finally, we calculate the time when object appear and exit the frame. We save that in a dataframe and after that we will visualize that dataframe.

In [ ]:
import cv2, time, pandas
from datetime import datetime

first_frame = None
status_list = [None,None]
time = []
df = pandas.DataFrame(columns=["start","End"])

video = cv2.VideoCapture(0) #Conver the video capture object to record video using webcam

while True:
    check , frame = video.read()
    status = 0
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Convert the frame color to gray scale
    gray = cv2.GaussianBlur(gray, (21,21),0) #Convert the gray scale frme to GaussianBlur image

    #This is used to store the first image of the video
    if first_frame is None:
        first_frame = gray

    #Delta frame
    #Compare first frame to current frame
    delta_frame = cv2.absdiff(first_frame, gray)
    thresh_delta = cv2.threshold(delta_frame, 30, 255, cv2.THRESH_BINARY)[1]
    thresh_delta = cv2.dilate(thresh_delta, None, iterations=0)
    
    #Contours
    cnts, hierarchy = cv2.findContours(thresh_delta.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)#Draw rectangle for moving objects
    #(_, cnts, _) = cv2.findContours(thresh_delta.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #define the contour arear. Basically , add the border

    #Draw rectangle for moving objects
    for contour in cnts:
        if cv2.contourArea(contour) < 1000:
            continue
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3)

    #Show the captured frame from video in a window
    cv2.imshow('frame', frame)
    cv2.imshow('Capturing',gray)
    cv2.imshow('delta',delta_frame)
    cv2.imshow('thresh',thresh_delta)
    
    #Set keyborad key q=quit
    key = cv2.waitKey(1)
    if(key == ord('q')):
        break
        

video.release()
cv2.destroyAllWindows()


# Record Motion timing

In [11]:
import cv2, time
import pandas as pd
from datetime import datetime

first_frame = None
status_list = [None,None]
times = []
df = pd.DataFrame(columns=["Start","End"])

video = cv2.VideoCapture(0) #Conver the video capture object to record video using webcam

while True:
    check , frame = video.read()
    status = 0
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Convert the frame color to gray scale
    gray = cv2.GaussianBlur(gray, (21,21),0) #Convert the gray scale frme to GaussianBlur image

    #This is used to store the first image of the video
    if first_frame is None:
        first_frame = gray

    #Delta frame
    #Compare first frame to current frame
    delta_frame = cv2.absdiff(first_frame, gray)
    thresh_delta = cv2.threshold(delta_frame, 30, 255, cv2.THRESH_BINARY)[1]
    thresh_delta = cv2.dilate(thresh_delta, None, iterations=0)
    
    #Contours
    cnts, hierarchy = cv2.findContours(thresh_delta.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)#Draw rectangle for moving objects
    #(_, cnts, _) = cv2.findContours(thresh_delta.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #define the contour arear. Basically , add the border

    #Draw rectangle for moving objects
    for contour in cnts:
        if cv2.contourArea(contour) < 1000:
            continue
        status = 1
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3)
        
        
    # Motion time
    status_list.append(status) #Capture staring Time

    if status_list[-1]==1 and status_list[-2]==0: #It is a 1st frame
        times.append(datetime.now())
    if status_list[-1]==0 and status_list[-2]==1: #Its is a last frame
        times.append(datetime.now())

    #Show the captured frame from video in a window
    cv2.imshow("First Frame", first_frame)
    cv2.imshow('Caturing frames', frame)
    cv2.imshow('Capturing',gray)
    cv2.imshow('delta',delta_frame)
    cv2.imshow('thresh',thresh_delta)
    
    
    
    
    #Set q=quit
    key = cv2.waitKey(1)
    if(key == ord('q')):
        if status is 1:
            times.append(datetime.now())
        break
        
print(status_list)
print(times)

#Store into DataFrame
for i in range(0, len(times), 2):
    df = df.append({
    "Start":times[i],
    "End":times[i+1]},
    ignore_index = True)

df.to_csv(input_path+"Times.csv", index = False)

video.release()
cv2.destroyAllWindows()

[None, None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[datetime.datetime(2020, 6, 12, 11, 41, 21, 677300), datetime.datetime(2020, 6, 12, 11, 41, 21, 923146), datetime.datetime(2020, 6, 12, 11, 41, 22, 63135), datetime.datetime(2020, 6, 12, 11, 41, 34, 939890)]


# Plotting Motion Detection Graph

In [5]:
#bokeh is a library that help us to visulaize  data on browser
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool, ColumnDataSource

In [17]:
df["Start_string"] = df["Start"].dt.strftime("%Y-%m-%d %H:%M:%S")
df["End_string"] =  df["End"].dt.strftime("%Y-%m-%d %H:%M:%S")


cds=ColumnDataSource(df)

p=figure(x_axis_type='datetime', height=100, width=500,  title="Motion Grpah")
p.yaxis.minor_tick_line_color = None
p.ygrid[0].ticker.desired_num_ticks=1

hover=HoverTool(tooltips=[("Start","@Start_string"), ("End", "@End_string")])
p.add_tools(hover)

q=p.quad(left="Start", right="End", bottom=0, top=1, color="red", source=cds)


output_file(input_path+"Graph.html")
show(p)